In [4]:
import pandas as pd
from pprint import pprint
from util.DatasetManager import DatasetManager
from util.DataframeManager import DataframeManager

# Variables used all over the notebook and *not changed*

In [5]:
DATASET_ID = "T004"
datasetManager = DatasetManager(DATASET_ID)

# Opening the dataset and getting general info

In [6]:
filename = "T004_new-road-vehicle-registrations-by-vehicle-category-and-fuel-type.csv"
path = "/Users/hlinero/Desktop/iTEM Material/raw dataset/T004/{}".format(filename)
df = datasetManager.get_dataframe_from_csv_file(path, ";")
df

NameError: name 'pd' is not defined